In [28]:
# --- 1. ADIM: GROQ KURULUM VE BAĞLANTI ---
!pip install -q groq

import os
from google.colab import userdata
from groq import Groq
import re
import time
import json

# API Bağlantısı
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    client = Groq(api_key=groq_api_key)
    print("✅ Groq Bağlantısı Hazır! (Model: Qwen 2.5 32B)")
except Exception as e:
    print(f"⚠️ HATA: GROQ_API_KEY bulunamadı. Secrets kısmına ekledin mi? ({e})")

✅ Groq Bağlantısı Hazır! (Model: Qwen 2.5 32B)


In [19]:
# --- GÜNCELLENMİŞ VE DAHA AKILLI RAG MOTORU ---
class SimpleRAG:
    def __init__(self):
        self.vector_store = []

    def get_embedding(self, text):
        words = re.findall(r'\w+', text.lower())
        return Counter(words)

    def _cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum(vec1[x] * vec2[x] for x in intersection)
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        return float(numerator) / denominator if denominator else 0.0

    def ingest(self, text, source):
        if text.strip():
            self.vector_store.append({
                "content": text.strip(),
                "vector": self.get_embedding(text),
                "source": source
            })


    def retrieve(self, query, top_k=3):
        query_vec = self.get_embedding(query)
        query_tokens = query.lower().split()
        scores = []

        for item in self.vector_store:

            score = self._cosine_similarity(query_vec, item["vector"])
            content_lower = item["content"].lower()


            for token in query_tokens:
                if token in content_lower:
                    if any(char.isdigit() for char in token):
                        score += 2.0  # Kritik model numarası bonusu
                    elif len(token) > 2:
                        score += 0.2  # Normal kelime bonusu (MSI, MAG vb.)

            scores.append((score, item["content"]))

        scores.sort(key=lambda x: x[0], reverse=True)
        return "\n".join([item[1] for item in scores[:top_k] if item[0] > 0.1])

In [20]:
# --- EKSİK OLAN PARÇA: RAG MOTORUNU BAŞLATMA VE VERİ YÜKLEME ---
import json

# 1. Motoru Oluştur
rag_engine = SimpleRAG()

# 2. Veritabanı Dosyasını Yükle
try:
    with open("hardware_database_FULL.json", "r", encoding="utf-8") as f:
        database_data = json.load(f)

    # 3. Verileri Motora Besle (Ingest)
    print("📥 Veritabanı yükleniyor...")
    count = 0
    for item in database_data:
        # Veriyi metne çevirip vektör veritabanına atıyoruz
        # Örn: "gpu - RTX 4090: {'tdp': 450, ...}"
        content = f"{item['category']} - {item['model']}: {json.dumps(item['specs'])}"
        rag_engine.ingest(content, source="JSON_DB")
        count += 1

    print(f"✅ RAG Motoru Hazır! Toplam {count} parça yüklendi.")
    print("🚀 Artık 'rag_engine' tanımlı, benchmarkı tekrar başlatabilirsin.")

except FileNotFoundError:
    print("🚨 HATA: 'hardware_database_FULL.json' dosyası bulunamadı! Lütfen dosyayı yüklediğinden emin ol.")

📥 Veritabanı yükleniyor...
✅ RAG Motoru Hazır! Toplam 335 parça yüklendi.
🚀 Artık 'rag_engine' tanımlı, benchmarkı tekrar başlatabilirsin.


In [21]:

def donanim_arama(query: str) -> str:

    # Giriş temizliği
    clean_query = query.strip().strip("-").strip()
    print(f"🔍 İNCELENİYOR: '{clean_query}'")

    result = rag_engine.retrieve(clean_query)


    if not result:
        return "Veritabanında bu parça ile ilgili bilgi bulunamadı."
    return result


system_prompt = """
Sen uzman bir PC Toplama Mühendisisin.
Soruları şu döngüyle çöz: Düşünce (Thought) -> Eylem (Action) -> Gözlem (Observation).

ÇOK ÖNEMLİ KURALLAR (DÖNGÜ ENGELLEME VE HİBRİT ZEKA):
1. ARAMA KURALI (ANTI-LOOP): Bir parça için SADECE BİR KEZ arama yap.
   - Eğer 'donanim_arama' sonucu "Bulunamadı" derse veya boş gelirse, ASLA TEKRAR ARAMA.
   - Hemen 'Thought' adımına geç ve "Veritabanında yok, genel bilgimi (hafızamı) kullanıyorum" de.

2. HİBRİT CEVAPLAMA:
   - Önce veritabanına bak (RAG).
   - Veritabanında varsa -> O veriyi kullan.
   - Veritabanında yoksa -> Pes etme! Kendi genel bilgini (LLM Knowledge) kullan.
   - Ancak cevabın başına "Kaynak: Genel Bilgi" notunu düş.
   - Cevabını "Cevap:" diyerek hemen bitir. Uzatma.

3. MANTIK KURALLARI:
   - PSU Hesabı: (GPU TDP + CPU TDP + 100W Sistem Payı) > PSU Watt ise YETERSİZ de.
   - Soket Uyumu: İşlemci Soketi ile Anakart Soketi AYNI ise UYUMLU de.

4. Arama Yaparken: Parçaları TEK TEK ara. Asla "ve" ile birleştirme.

Örnek (Veride Olmayan Parça İçin):
Action: donanim_arama GTX 1080
Observation: Veritabanında bilgi bulunamadı.
Thought: Veritabanında yok, kural gereği tekrar arayamam. Hafızamdan biliyorum ki TDP'si 180W.
Cevap: Kaynak: Genel Bilgi. GTX 1080 TDP değeri 180W'tır.
""".strip()

In [29]:
import re
import time
import google.generativeai as genai

# --- GLOBAL TANIMLAMALAR (Regex ve Aksiyonlar) ---
# Bu kısımların fonksiyonlardan önce tanımlı olduğundan emin olmalıyız
action_re = re.compile(r'(?:Action|Eylem|İşlem)?\s*:?\s*(donanim_arama)(?:[\s:("]+)(.*?)(?:[)"]*)\s*$', re.IGNORECASE | re.MULTILINE)
known_actions = {"donanim_arama": donanim_arama} # donanim_arama fonksiyonunun yukarıda tanımlı olduğunu varsayıyoruz


class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        # System prompt'u mesaj geçmişine ekle
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        # Kullanıcı mesajını ekle
        self.messages.append({"role": "user", "content": message})
        return self.execute()

    def execute(self):
        try:
            # GROQ API ÇAĞRISI
            completion = client.chat.completions.create(
                model="qwen-2.5-32b",  # <-- İŞTE QWEN BURADA
                messages=self.messages,
                temperature=0.1,
                max_tokens=2048,
                stop=["Observation:", "Gözlem:", "PAUSE"] # Modelin durması gereken yerler
            )
            # Cevabı al
            response_text = completion.choices[0].message.content

            # Asistan cevabını geçmişe ekle (Sohbet devamlılığı için)
            self.messages.append({"role": "assistant", "content": response_text})

            return response_text

        except Exception as e:
            # Hata durumunda
            print(f"\n🚨 API HATASI: {e}\n")
            return f"API Hatası: {e}"

# --- BENCHMARK DÖNGÜSÜ (DÜZELTİLMİŞ) ---
def run_benchmark_query(question, max_turns=7):
    i = 0
    react_agent = Agent(system_prompt)
    next_prompt = question
    final_answer = "Cevap bulunamadı (Döngü Limiti)."

    print(f"   ↪️ Düşünmeye başladı...")

    while i < max_turns:
        i += 1
        result = react_agent(next_prompt)

        # Hata Kontrolünü Düzelttik (String içinde arama yapıyoruz)
        if "API Hatası" in result or "HATA:" in result:
            return f"İşlem Başarısız: {result}"

        # Debug: Modelin ne dediğini görelim
        print(f"   [Adım {i}] Model: {result[:100].replace(chr(10), ' ')}...")

        matches = list(action_re.finditer(result))

        if matches:
            match = matches[-1]
            action, action_input = match.groups()
            action_input = action_input.strip().strip('"').strip("'").strip("-").strip()

            if action.lower() in known_actions:
                print(f"   ⚙️ Aksiyon: {action} -> {action_input}")
                # Gerçek Python fonksiyonunu çalıştır
                observation = known_actions[action.lower()](action_input)
                next_prompt = f"Observation: {observation}"
            else:
                next_prompt = "Observation: Böyle bir araç yok."
        else:
            if "Cevap:" in result:
                final_answer = result.split("Cevap:")[-1].strip()
                return final_answer
            elif "Final Answer:" in result:
                final_answer = result.split("Final Answer:")[-1].strip()
                return final_answer
            else:
                if i == max_turns:
                    return "Cevap Bulunamadı (Zaman Aşımı)"

    return final_answer

In [30]:
# =============================================================================
# BENCHMARK SORU SETİ (GÜNCELLENMİŞ)
# =============================================================================

benchmark_questions = [
    # --- BÖLÜM 1: PC DONANIM & TEKNİK BİLGİ (Modelin Genel Kültürü) ---
    # Bu soruların cevabı senin RAG veritabanında yok, modelin kendi bilgisini test ediyoruz.

    {"id": 1, "kategori": "Genel Bilgi", "soru": "DLSS teknolojisi nedir ve ne işe yarar?"},
    {"id": 2, "kategori": "Genel Bilgi", "soru": "İşlemciye termal macun sürmezsem ne olur?"},
    {"id": 3, "kategori": "Genel Bilgi", "soru": "M.2 NVMe SSD ile SATA SSD arasındaki temel fark nedir?"},
    {"id": 4, "kategori": "Genel Bilgi", "soru": "RAM'lerdeki CL değeri (Gecikme) düşük mü olmalı yüksek mi?"},
    {"id": 5, "kategori": "Genel Bilgi", "soru": "Sıvı soğutma mı yoksa hava soğutma mı daha güvenlidir?"},
    {"id": 6, "kategori": "Genel Bilgi", "soru": "Bir monitörde IPS panel mi yoksa TN panel mi daha iyi renk verir?"},
    {"id": 7, "kategori": "Genel Bilgi", "soru": "Overclock (Hız aşırtma) garantiyi bozar mı?"},
    {"id": 8, "kategori": "Genel Bilgi", "soru": "Modüler PSU ile modüler olmayan PSU arasındaki fark nedir?"},
    {"id": 9, "kategori": "Genel Bilgi", "soru": "BIOS güncellemesi yapmak performansı artırır mı?"},
    {"id": 10, "kategori": "Genel Bilgi", "soru": "Ekran kartında Ray Tracing (Işın İzleme) ne demektir?"},

    # --- BÖLÜM 2: MANTIK VE UYUMLULUK (Reasoning) ---
    # Modelin donanım parçaları arasındaki fiziksel/mantıksal ilişkiyi bilmesi gerekir.

    {"id": 11, "kategori": "Mantık - Uyum", "soru": "Micro-ATX bir anakart, ATX boyutundaki bir kasaya takılabilir mi?"},
    {"id": 12, "kategori": "Mantık - Uyum", "soru": "DDR4 RAM'i DDR5 slotuna zorlayarak takabilir miyim?"},
    {"id": 13, "kategori": "Mantık - Uyum", "soru": "PCIe 4.0 ekran kartı, PCIe 3.0 anakartta çalışır mı?"},
    {"id": 14, "kategori": "Mantık - Darboğaz", "soru": "Çok güçlü bir ekran kartı ile çok zayıf bir işlemci kullanırsam ne olur?"},
    {"id": 15, "kategori": "Mantık - Soğutma", "soru": "Kasa fanlarının hepsi içeri üflerse hava akışı nasıl olur?"},
    {"id": 16, "kategori": "Mantık - Görüntü", "soru": "60Hz monitörde oyun 200 FPS verirse ne olur, akıcılık artar mı?"},
    {"id": 17, "kategori": "Mantık - Güç", "soru": "80 Plus Gold sertifikası PSU'nun daha fazla elektrik ürettiğini mi gösterir?"},
    {"id": 18, "kategori": "Mantık - Bağlantı", "soru": "DisplayPort kablosu ses iletir mi?"},
    {"id": 19, "kategori": "Mantık - RAM", "soru": "Bilgisayara 3 tane RAM takarsam Dual Channel çalışır mı?"},
    {"id": 20, "kategori": "Mantık - Depolama", "soru": "Hard disk (HDD) oyun yükleme sürelerini SSD'ye göre nasıl etkiler?"},

    # --- BÖLÜM 3: SENARYO VE TAVSİYE (Veritabanı Dışı) ---
    # Kullanıcı tavsiye istiyor, model yorum yapmalı.

    {"id": 21, "kategori": "Senaryo", "soru": "Sadece League of Legends oynayacağım, harici ekran kartı şart mı yoksa APU yeter mi?"},
    {"id": 22, "kategori": "Senaryo", "soru": "Yayıncı (Streamer) olmak istiyorum, işlemci mi daha önemli ekran kartı mı?"},
    {"id": 23, "kategori": "Senaryo", "soru": "Video kurgu (Render) için en az kaç GB RAM önerirsin?"},
    {"id": 24, "kategori": "Senaryo", "soru": "RGB ışıklandırma bilgisayarın performansını (FPS) artırır mı?"},
    {"id": 25, "kategori": "Senaryo", "soru": "İkinci el ekran kartı alırken 'Mining' yapılıp yapılmadığı neden önemlidir?"},
    {"id": 26, "kategori": "Senaryo", "soru": "Laptop işlemcisi sökülüp masaüstü bilgisayara takılır mı?"},
    {"id": 27, "kategori": "Senaryo", "soru": "Bilgisayar açılırken anakarttan 'bip' sesleri gelmesi ne anlama gelir?"},
    {"id": 28, "kategori": "Senaryo", "soru": "Server (Sunucu) RAM'i normal ev bilgisayarında çalışır mı?"},
    {"id": 29, "kategori": "Senaryo", "soru": "Windows 11 kurmak için TPM 2.0 şart mı?"},
    {"id": 30, "kategori": "Senaryo", "soru": "Ekran kartım çok ısınıyor, yan kapağı açmak mantıklı mı?"},

    # --- BÖLÜM 4: ESKİ NESİL DONANIM (Tuzak Sorular) ---
    # Modelin eskiyi yeniyi ayırt etmesi lazım.

    {"id": 31, "kategori": "Eski Nesil", "soru": "Nvidia GTX 1080 Ti ekran kartı günümüz oyunları için hala yeterli mi?"},
    {"id": 32, "kategori": "Eski Nesil", "soru": "Intel i7-4790K işlemcisi DDR4 RAM destekler mi?"},
    {"id": 33, "kategori": "Eski Nesil", "soru": "AMD FX serisi işlemciler çok ısınıyor muydu?"},
    {"id": 34, "kategori": "Eski Nesil", "soru": "AGP slotlu ekran kartımı yeni anakarta takabilir miyim?"},
    {"id": 35, "kategori": "Eski Nesil", "soru": "IDE kablosu ile SATA kablosu aynı mıdır?"},
    {"id": 36, "kategori": "Eski Nesil", "soru": "Windows XP hala güvenli bir işletim sistemi mi?"},
    {"id": 37, "kategori": "Eski Nesil", "soru": "Floppy disk sürücüsü yeni kasalara takılır mı?"},
    {"id": 38, "kategori": "Eski Nesil", "soru": "SLI veya Crossfire yapmak 2025 yılında mantıklı mı?"},
    {"id": 39, "kategori": "Eski Nesil", "soru": "Voodoo ekran kartları hala üretiliyor mu?"},
    {"id": 40, "kategori": "Eski Nesil", "soru": "Tüplü (CRT) monitörlerin LCD'ye göre avantajı var mıydı?"},

    # --- BÖLÜM 5: ALAKASIZ / UÇ ÖRNEKLER (Out of Domain) ---
    # Modelin "Bu benim alanım değil" demesi veya sadece bilgiyi verip PC ile alakasız olduğunu belli etmesi lazım.

    {"id": 41, "kategori": "Alakasız", "soru": "iPhone 15 Pro Max kamerasının diyafram açıklığı nedir?"},
    {"id": 42, "kategori": "Alakasız", "soru": "PlayStation 5 Slim ile Xbox Series X arasındaki fark nedir?"},
    {"id": 43, "kategori": "Alakasız", "soru": "Samsung Galaxy S24 Ultra işlemcisi Snapdragon mu Exynos mu?"},
    {"id": 44, "kategori": "Alakasız", "soru": "Tesla Model Y'nin menzili kaç kilometredir?"},
    {"id": 45, "kategori": "Alakasız", "soru": "Makarna en iyi kaç dakikada haşlanır?"},
    {"id": 46, "kategori": "Alakasız", "soru": "Dyson V15 süpürgenin emiş gücü kaç AW?"},
    {"id": 47, "kategori": "Alakasız", "soru": "Nintendo Switch OLED modelinin ekran boyutu nedir?"},
    {"id": 48, "kategori": "Alakasız", "soru": "Türkiye'nin başkenti neresidir?"},
    {"id": 49, "kategori": "Alakasız", "soru": "Kahve makinesinin kireci nasıl temizlenir?"},
    {"id": 50, "kategori": "Alakasız", "soru": "Klima alırken BTU hesabı nasıl yapılır?"}
]



In [31]:
# --- 3. ADIM: TESTİ BAŞLAT (QWEN İÇİN DÜZELTİLDİ) ---
import json
import time

# 1. İSİM GÜNCELLEMESİ (Dosya adı karışmasın)
MODEL_NAME = "Groq_Qwen_2.5_32B_Benchmark"

results = []

print(f"🚀 BENCHMARK BAŞLIYOR: {MODEL_NAME}")
print("="*60)

# benchmark_questions listesinin hafızada olduğunu varsayıyoruz
for item in benchmark_questions:
    print(f"\nSoru {item['id']} [{item['kategori']}]: {item['soru']}")

    start_time = time.time()

    try:
        # run_benchmark_query, bir önceki hücrede tanımladığımız Groq Agent'ı kullanacak
        cevap = run_benchmark_query(item['soru'])
    except Exception as e:
        cevap = f"HATA: {str(e)}"

    end_time = time.time()
    duration = round(end_time - start_time, 2)

    print(f"🤖 Qwen Cevabı: {cevap}")
    print(f"⏱️ Süre: {duration} sn")
    print("-" * 40)

    results.append({
        "id": item["id"],
        "kategori": item["kategori"],
        "soru": item["soru"],
        "model_cevabi": cevap,
        "sure_sn": duration,
        "model_adi": MODEL_NAME
    })

    # Groq ücretsiz API'si bazen seri isteklerde hata verebilir,
    # o yüzden her sorudan sonra 1-2 saniye beklemek çok sağlıklıdır.
    time.sleep(2)

# Sonuçları Kaydet
dosya_adi = f"benchmark_sonuclari_{MODEL_NAME}.json"
with open(dosya_adi, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"\n✅ TÜM TESTLER BİTTİ! Sonuçlar '{dosya_adi}' dosyasına kaydedildi.")

🚀 BENCHMARK BAŞLIYOR: Groq_Qwen_2.5_32B_Benchmark

Soru 1 [Genel Bilgi]: DLSS teknolojisi nedir ve ne işe yarar?
   ↪️ Düşünmeye başladı...

🚨 API HATASI: Error code: 400 - {'error': {'message': 'The model `qwen-2.5-32b` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

🤖 Qwen Cevabı: İşlem Başarısız: API Hatası: Error code: 400 - {'error': {'message': 'The model `qwen-2.5-32b` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}
⏱️ Süre: 0.17 sn
----------------------------------------

Soru 2 [Genel Bilgi]: İşlemciye termal macun sürmezsem ne olur?
   ↪️ Düşünmeye başladı...

🚨 API HATASI: Error code: 400 - {'error': {'message': '

KeyboardInterrupt: 